In [1]:
import numpy as np
import pandas as pd
import os
import re

# Data Prep for Model

In [2]:
state_level_df = pd.read_csv('data/cln/all_states_real_sir.csv')
state_level_df[['state','population']].drop_duplicates()
state_level_df = state_level_df[state_level_df['state'].isin(['OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'])]

state_level_df['standardized_date'] = pd.to_datetime(state_level_df['standardized_date'])
start_date_df = state_level_df.groupby(['state'],as_index = False).agg({'standardized_date':'min'})
start_date_df.columns = ['state','start_date']

state_level_df = pd.merge(state_level_df,start_date_df)
state_level_df['day'] = state_level_df['standardized_date'] - state_level_df['start_date']
state_level_df['day'] = state_level_df['day'].apply(lambda x : x.days)
state_level_df[state_level_df['day'] == 0]

,state,standardized_date,# Infected,# Deaths,total_number_of_days,population,# Susceptible,start_date,day
0,OH,2020-03-13,13.0,0.0,31,11536504,11536491.0,2020-03-13,0
32,OK,2020-03-16,10.0,0.0,28,3751351,3751341.0,2020-03-16,0
61,OR,2020-03-08,14.0,0.0,36,3831074,3831060.0,2020-03-08,0
98,PA,2020-03-09,12.0,0.0,35,12702379,12702367.0,2020-03-09,0
134,RI,2020-03-13,14.0,0.0,31,1052567,1052553.0,2020-03-13,0
166,SC,2020-03-10,10.0,0.0,34,4625364,4625354.0,2020-03-10,0
201,SD,2020-03-16,10.0,1.0,28,814180,814170.0,2020-03-16,0
230,TN,2020-03-12,17.0,0.0,32,6346105,6346088.0,2020-03-12,0
263,TX,2020-03-08,11.0,0.0,36,25145561,25145550.0,2020-03-08,0
300,UT,2020-03-14,19.0,0.0,30,2763885,2763866.0,2020-03-14,0


In [3]:
state_level_df.columns=['Region name', 'date','# Infected','# Removed', 'total_number_of_days', 
                        'population','# Susceptible','start_date','day']
test_data = state_level_df[['Region name', 'date','# Infected','# Removed','# Susceptible','day']].copy()

test_data.head(3)

,Region name,date,# Infected,# Removed,# Susceptible,day
0,OH,2020-03-13,13.0,0.0,11536491.0,0
1,OH,2020-03-14,25.0,0.0,11536479.0,1
2,OH,2020-03-15,37.0,0.0,11536467.0,2


In [4]:
total_day_state = state_level_df[['Region name', 'total_number_of_days']].drop_duplicates()
total_day_state

,Region name,total_number_of_days
0,OH,31
32,OK,28
61,OR,36
98,PA,35
134,RI,31
166,SC,34
201,SD,28
230,TN,32
263,TX,36
300,UT,30


# Model Starts here


In [5]:
from src.eval_model import *
from src.SIR import sir, dynamic_sim_sir, dynamic_sim_sir_df

# SIR: Find best $I_0$, beta, gamma 

- Optimize the SIR model first
- Using the first 15 day's of data, rest days will be used for evaluation
- Allow inital infected number of be adjust up

In [6]:
# specify range

I_adjust_range = [1]# I0 = I * I_range_adjust
beta_range = np.arange(0.5e-8, 5e-7, 3e-9)
beta_decay = [0.005,0.0075, 0.01, 0.025, 0.05,0.075,0.1,0.125, 0.15]
beta_decay_start_day = [5,10,15,20]
gamma_range = [1/x for x in range(4, 24, 4)]


def print_range(txt, x):
    print(txt + ' : ' + str(min(x)) + ' - ' + str(max(x)) + ' (' + str(len(x)) + ')')

print_range('# Infected Adjustment', I_adjust_range)
print_range('beta', beta_range)
print_range('beta_decay', beta_decay)
print_range('beta_decay_start_day', beta_decay_start_day)
print_range('gamma', gamma_range)
print_range('1/gamma', [1/x for x in gamma_range])

# Infected Adjustment : 1 - 1 (1)
beta : 5e-09 - 4.970000000000001e-07 (165)
beta_decay : 0.005 - 0.15 (9)
beta_decay_start_day : 5 - 20 (4)
gamma : 0.05 - 0.25 (5)
1/gamma : 4.0 - 20.0 (5)


In [7]:
I_adjust_range_v, beta_range_v, beta_decay_v, beta_decay_start_day_v, gamma_v = np.meshgrid(
    I_adjust_range,\
    beta_range, \
    beta_decay,\
    beta_decay_start_day,\
    gamma_range,\
    sparse = False)

Grid = pd.DataFrame({
    'I_adjust':I_adjust_range_v.flatten(),
    'beta': beta_range_v.flatten(),
    'beta_decay': beta_decay_v.flatten(),
    'beta_decay_start_day': beta_decay_start_day_v.flatten(),
    'gamma':gamma_v.flatten()
})

In [8]:
Grid.shape

(29700, 5)

In [9]:
# get the day 0's number

DAY_0 = test_data[test_data['day'] == 0].copy()

DAY_0['N'] = DAY_0['# Infected'] + DAY_0['# Susceptible'] + DAY_0['# Removed']

DAY_0

,Region name,date,# Infected,# Removed,# Susceptible,day,N
0,OH,2020-03-13,13.0,0.0,11536491.0,0,11536504.0
32,OK,2020-03-16,10.0,0.0,3751341.0,0,3751351.0
61,OR,2020-03-08,14.0,0.0,3831060.0,0,3831074.0
98,PA,2020-03-09,12.0,0.0,12702367.0,0,12702379.0
134,RI,2020-03-13,14.0,0.0,1052553.0,0,1052567.0
166,SC,2020-03-10,10.0,0.0,4625354.0,0,4625364.0
201,SD,2020-03-16,10.0,1.0,814170.0,0,814181.0
230,TN,2020-03-12,17.0,0.0,6346088.0,0,6346105.0
263,TX,2020-03-08,11.0,0.0,25145550.0,0,25145561.0
300,UT,2020-03-14,19.0,0.0,2763866.0,0,2763885.0


In [10]:
def get_vetcor_beta(p_original_beta,p_decay, p_start_day, p_total_day):
    p_start_day=int(p_start_day)
    p_total_day= int(p_total_day)
    new_beta_v=[]
    if p_start_day>p_total_day:
        return [p_original_beta]*p_total_day
    else:
        for day in range(p_start_day):
            new_beta_v.append(p_original_beta)
        for day in range(p_start_day,p_total_day):
            new_beta_v.append(new_beta_v[-1]*(1-p_decay))
        return new_beta_v
        

In [ ]:
# grid search for each region

region_var = 'Region name'
n_days_eval = 0 # carve out 5 days for back test

for region in test_data[region_var].unique():
    
    print(region)
    
    simulation_results = pd.DataFrame()
    
    for index, row in Grid.iterrows():
    
        # use part of days to do grid search
        n_days = total_day_state.loc[total_day_state[region_var] == region, 
                                 'total_number_of_days'].tolist()[0] - n_days_eval
        
        # Get parameter from Grid
        i_adjust = row['I_adjust']
        gamma = row['gamma']
        beta_vec = get_vetcor_beta(row['beta'], row['beta_decay'], row['beta_decay_start_day'],n_days)

        # fixed
        N =  DAY_0.loc[DAY_0[region_var] == region,'N'].tolist()[0]
        R =  DAY_0.loc[DAY_0[region_var] == region,'# Removed'].tolist()[0]

        # adjust I
        I =  int(DAY_0.loc[DAY_0[region_var] == region,'# Infected'].tolist()[0] * i_adjust)

        # back calculcate S
        S = N - I - R

        report = dynamic_sim_sir_df(S, I, R, gamma, 0, beta_vec, n_days)
        report[region_var] = region

        # include parameter setting
        report['beta'] = row['beta']
        report['beta_decay'] = row['beta_decay']
        report['beta_decay_start_day'] = row['beta_decay_start_day']
        report['gamma'] = gamma
        report['I0'] = I

        # append
        simulation_results = simulation_results.append(report)
    
    simulation_results = pd.merge(simulation_results, test_data, on = [region_var, 'day'])
    
    print(simulation_results.shape)

    simulation_results.to_csv('result/sir_decay_states_part2/grid/' + region + '.csv', index = False)


OH


In [ ]:
best_param = pd.DataFrame()
grid_path = 'result/sir_decay_states_part2/grid/'

for path in os.listdir(grid_path):
    
    if re.findall(".csv", path):
    
        region = re.sub(".csv", "", path)
    
        data = pd.read_csv(grid_path + path)
    
        data = data.groupby(['beta', 'beta_decay','beta_decay_start_day','gamma', 'I0']).\
        apply(lambda x: gen_Metrics(x, 'infected', '# Infected')).\
        reset_index().\
        drop('level_5', axis = 1)
    
        tmp = data.loc[data['SSE'].idxmin()].copy() ## min
    
        tmp[region_var] = region
    
        best_param = best_param.append(tmp)

best_param.to_excel(grid_path + '_best_param.xlsx', index = False)

In [ ]:
best_param
# 3-30

# Final Model

### SIR model

In [ ]:
region_var = 'Region name'
n_days_fit = 30 
best_param_1 = pd.read_excel('result/sir_decay_states_part2/grid/_best_param.xlsx')
final_result = pd.DataFrame()

for region in DAY_0[region_var]:
    
    # fixed
    N =  DAY_0.loc[DAY_0[region_var] == region,'N'].tolist()[0]
    R =  DAY_0.loc[DAY_0[region_var] == region,'# Removed'].tolist()[0]
    
    # best SIR 
    I = best_param_1.loc[best_param_1[region_var] == region, 'I0'].tolist()[0]
    beta = best_param_1.loc[best_param_1[region_var] == region, 'beta'].tolist()[0]
    beta_decay = best_param_1.loc[best_param_1[region_var] == region, 'beta_decay'].tolist()[0]
    beta_decay_start_day = best_param_1.loc[best_param_1[region_var] == region, 'beta_decay_start_day'].tolist()[0]
    gamma = best_param_1.loc[best_param_1[region_var] == region, 'gamma'].tolist()[0]
                
    S = N - I - R
    
    # use all available days
    n_days = total_day_state.loc[total_day_state[region_var] == region, 
                                 'total_number_of_days'].tolist()[0]
    
    beta_vec = get_vetcor_beta(beta, beta_decay, beta_decay_start_day,n_days)
            
    report = dynamic_sim_sir_df(S, I, R, gamma, 0, beta_vec, n_days)
    report[region_var] = region
                
    # append
    report['day'] = report.index.astype(int)
    final_result = final_result.append(report)


final_result.to_excel('result/sir_decay_states_part2/final_sir_model.xlsx', index = False)

# Plot the curve

In [ ]:
final_result_sir_decay = pd.read_excel('result/sir_decay_states_part2/final_sir_model.xlsx')\
[['susceptible', 'infected', 'recovered', 'Region name', 'day']].\
rename(columns = {'susceptible':'SIR Decay # Susceptible',
                 'infected':'SIR Decay # Infected',
                 'recovered':'SIR Decay # Removed'})

final = pd.merge(test_data, final_result_sir_decay)

min_date = min(final['date'])
max_date = max(final['date'])

min_y = min(min(final.loc[final['Region name'].isin(['NY', 'MI']) == False, 'SIR Decay # Infected']),
            min(final.loc[final['Region name'].isin(['NY', 'MI']) == False, '# Infected']))

max_y = max(max(final.loc[final['Region name'].isin(['NY', 'MI']) == False, 'SIR Decay # Infected']), 
            max(final.loc[final['Region name'].isin(['NY', 'MI']) == False, '# Infected']))
nrow = 10
ncol = 3

x = np.arange(0, nrow, 1)
y = np.arange(0, ncol, 1)
xx, yy = np.meshgrid(x, y)    
num = 0 

import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows = nrow, ncols = ncol, figsize = (ncol * 6, nrow * 5))

for region in final['Region name'].unique():
    
    
    region_data = final[final['Region name'] == region].copy()
    
    # find the last day for gridsearch
    end_day = total_day_state.loc[total_day_state['Region name'] == region, "total_number_of_days"].tolist()[0] - 5
   
    end_date = pd.to_datetime(region_data.loc[region_data['day'] == end_day, "date"].tolist()[0])
    
    # axis set up 
    x = xx.flatten()[num]
    y = yy.flatten()[num]
            
    # plot
    tmp = region_data[['date', 'SIR Decay # Infected', '# Infected']].copy()
    
    tmp['date'] = tmp['date'].apply(lambda x: pd.to_datetime(x))
    
    tmp = tmp.set_index('date')
    
    if region in ['NY', 'MI']:
        ax = tmp.plot(ax = axes[x, y], title = region, rot = 60, xlim = (min_date, max_date))
    else:
        ax = tmp.plot(ax = axes[x, y], title = region, rot = 60, xlim = (min_date, max_date))

    #ax.axvline(end_date, color="grey", linestyle="--")
    
    num += 1

plt.tight_layout()
plt.savefig('result/sir_decay_states_part2/final_fitted_sir_decay.png', dpi = 220)